<a href="https://colab.research.google.com/github/chia-yu0225/work/blob/main/EX05_04_Reply_Template_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pyngrok flask_ngrok
!pip install -q flask line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 9.7 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
    ngrok.connect(5000)
    run_with_ngrok(app)

from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    TemplateMessage, ComfirmTemplate, MessageAction,
    CarouselTemplate,
    CarouselColumn,
    URIAction,
    PostbackAction

)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))
@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    line_bot_api = MessagingApi(api_client)

    action = event.message.text
    if action == 'confirm':
        reply = TemplateMessage(
            alt_text='這是確認視窗',
            template=ComfirmTemplate(
                text='你喜歡葬送的艾芙蓮嗎？'
                actions=[
                    MessageAction(label='喜歡', text='喜歡'),
                    MessageAction(label='不喜歡', text='不喜歡'),
                ]
            )
        )
    elif action == 'carousel':
        carousel_template = CarouselTemplate(
            columns=[
                CarouselColumn(
                    thumbnail_image_url='https://example.com/image1.jpg',
                    title='夏日大作戰',
                    text='這是商品1的描述',
                    actions=[
                        URIAction(label='查看商品', uri='https://example.com/product1'),
                        MessageAction(label='加入購物車', data='product1')
                    ]
                ),
                CarouselColumn(
                    thumbnail_image_url='https://example.com/image2.jpg',
                    title='夏日大作戰',
                    text='這是商品2的描述',
                    actions=[
                        URIAction(label='查看商品', uri='https://example.com/product2'),
                        MessageAction(label='加入購物車', data='product2')
                    ]
                )
            ]
       )
        reply = TemplateMessage(
            alt_text='這是輪播視窗',
            template=carousel_template
        )
    else:
        reply = TextMessage(text='Thanks')

    line_bot_api.reply_message(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[reply]
        )
    )
ngrok_start()
if __name__ == "__main__"
    app.run()